In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium 
from folium.plugins import HeatMap


In [ ]:
# import data
data = pd.read_csv('../data/curated/tlc_data/first_cleaned.csv')

In [ ]:
# Calculate correlations for hourly trip count
temp_hourly_corr = clean_df['TMP'].corr(clean_df['hourly_trip_count'])
vis_hourly_corr = clean_df['VIS'].corr(clean_df['hourly_trip_count'])
events_hourly_corr = np.log1p(clean_df['Number of Events']).corr(clean_df['hourly_trip_count'])

# Calculate correlations for daily trip count
temp_daily_corr = clean_df['TMP'].corr(clean_df['daily_trip_count'])
vis_daily_corr = clean_df['VIS'].corr(clean_df['daily_trip_count'])
events_daily_corr = clean_df['Number of Events'].corr(clean_df['daily_trip_count'])

# Create a DataFrame to hold the correlation results
correlation_matrix = pd.DataFrame({
    'Correlation': ['temp_hourly_corr', 'vis_hourly_corr', 'events_hourly_corr', 
                    'temp_daily_corr', 'vis_daily_corr', 'events_daily_corr'],
    'Value': [temp_hourly_corr, vis_hourly_corr, events_hourly_corr, 
              temp_daily_corr, vis_daily_corr, events_daily_corr]
})

# Display the correlation matrix
print(correlation_matrix)


### Daily demand

In [ ]:
# Group by 'pickup_date' and 'PUBorough' to get the sum of trips and average weather data
daily_demand = clean_df.groupby(['pickup_date', 'PUBorough']).agg({
    'daily_trip_count': 'sum',
    'TMP': 'mean',
    'VIS': 'mean',
    'Number of Events': 'sum'
}).reset_index()

# Rename the columns to match the desired output
daily_demand.rename(columns={
    'daily_trip_count': 'total_daily_trips',
    'TMP': 'avg_temperature',
    'VIS': 'avg_visibility',
    'Number of Events': 'total_events'
}, inplace=True)

# Round the average temperature and visibility to 1 decimal place
daily_demand['avg_temperature'] = daily_demand['avg_temperature'].round(1)
daily_demand['avg_visibility'] = daily_demand['avg_visibility'].round(1)

# Show the first 5 rows of the aggregated result
print(daily_demand.head(5))


### Daily Trips by borough

In [ ]:
# Aggregate the number of daily trips by PUBorough
agg_data_daily = clean_df.groupby('PUBorough').agg({
    'daily_trip_count': 'sum'
}).reset_index()

# Rename the columns to match the expected output
agg_data_daily.rename(columns={'PUBorough': 'borough', 'daily_trip_count': 'total_daily_trips'}, inplace=True)

# Display the first 5 rows of the aggregated data
print(agg_data_daily.head())

# Create a bar chart with a log scale
fig, ax = plt.subplots(figsize=(10, 6))

# Create the bar plot using Seaborn
sbs.barplot(x='borough', y='total_daily_trips', data=agg_data_daily, ax=ax)

# Set the y-axis to a logarithmic scale
ax.set_yscale('log')

# Set the title of the plot
ax.set_title('Total Daily Trips by Borough')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Show the plot
plt.show()


In [ ]:
# save the plot
fig.savefig("../plots/total_daily_trips_by_borough.png")

### Pickups for each day of week

In [ ]:
# Plot number of pick ups per day of the week
# Group by 'Day of Week' and count the number of trips
trips_per_day_df = clean_df.groupby('Day of Week').agg({'daily_trip_count': 'sum'}).reset_index()

In [ ]:
# Plot the number of trips per day of the week
plt.figure(figsize=(12, 6))
sbs.barplot(x='Day of Week', y='daily_trip_count', data=trips_per_day_df, order=days_order)
plt.title('Number of Trips Per Day of the Week')
plt.xlabel('Day of Week')
plt.ylabel('Number of Trips')
plt.grid(True)
plt.tight_layout()

In [ ]:
# save the plot
plt.savefig('../plots/number_of_trips_per_day_of_week.png')